In [35]:
from PreProcessingBASD_v2 import *
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import tensorflow as tf
import numpy as np
import pandas as pd , sys
import matplotlib.pyplot as plt
import scikitplot as skplt
from IPython.display import clear_output
from sklearn.metrics import auc , roc_auc_score
from sklearn.model_selection import train_test_split
sys.path.append("/home/advice/Python/SR/Custom/")
from RAdam import RAdamOptimizer
import seaborn as sns
import re , os
from ColumnMatch import MatchVariable
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [36]:
data = pd.read_csv("./../../../Data/kdd/uci/uci_creditcard-train-0.2-0.0.csv")
in_var = ["LIMIT_BAL", "SEX", "EDUCATION", "MARRIAGE", "AGE", "PAY_0", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6",
          "BILL_AMT1", "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6",
          "PAY_AMT1", "PAY_AMT2", "PAY_AMT3", "PAY_AMT4", "PAY_AMT5", "PAY_AMT6"]
target_var = ['default payment next month']
fac_var = [ 'SEX','EDUCATION','MARRIAGE',]
num_var = [i for i in in_var if not i in fac_var]
#in_var = num_var + fac_var
data[fac_var] = data[fac_var].astype("str")

In [37]:
cat_col = data.select_dtypes("object").columns.tolist()
cat_col
#onehot_data = pd.get_dummies(data , columns= cat_col)

['SEX', 'EDUCATION', 'MARRIAGE']

In [38]:
select_var = in_var + target_var + ["sep_idx"]
data = data[select_var]

In [39]:
train = data[data.sep_idx ==1 ]
valid = data[data.sep_idx ==0 ]
## 
train = data
# _ = train.pop("sep_idx")
# _ = valid.pop("sep_idx")

In [40]:
test = pd.read_csv("./../../../Data/kdd/uci/uci_creditcard-test-0.2-0.0.csv")
test[fac_var] = test[fac_var].astype("str")
# test = test.drop(["ID","sep_idx"],axis=1)
# test.reset_index(drop=True ,inplace=True)

In [41]:
train.shape, valid.shape, test.shape

((30000, 25), (8982, 25), (6004, 26))

In [42]:
class CategoryHandler(BaseEstimator, TransformerMixin,MatchVariable,SKDecorate) :
    def __init__(self, trans_info=None, method = None , in_var= None , num_var =None, 
                 target_var=None , dummy_na = True) :
        self.trans_info = trans_info
        self.method = method
        self.in_var = in_var
        self.dummy_na = dummy_na
        self.fac_var = [var for var in in_var if var not in num_var]
        self.num_var = num_var
        self.target_var = target_var

In [43]:
trans_info = {}
Steps = [
    ("NumericImpute", MissingHandling(method="mean", 
                               trans_info= trans_info,
                               var= num_var)) , 
    ("FactorImpute", MissingHandling(method="most_frequent", 
                               trans_info= trans_info,
                               var= fac_var)) , 
    ("numeric", NumericHandler(method="normal",
                               trans_info= trans_info,
                               num_var=num_var)),
]

pipe = Pipeline(Steps)
pipe.fit(train)

Transform MissingData Method : mean
Transform MissingData Method : most_frequent


Pipeline(memory=None,
         steps=[('NumericImpute',
                 MissingHandling(method=None,
                                 trans_info={'fill_value': array([ 1.67484323e+05,  3.54752228e+01, -1.70900886e-02, -1.25958973e-01,
       -1.65824567e-01, -2.16004663e-01, -2.64603095e-01, -2.86467414e-01,
        5.10673010e+04,  4.95652237e+04,  4.70681142e+04,  4.32569520e+04,
        4.04164970e+04,  3.88574464e+04,  5.69172658e+03,  5.81409563e+03...
        5.13448505e+03,  4.83411333e+03,  4.76632123e+03,  5.24146976e+03]),
                                            'norm_std': array([1.29747662e+05, 8.24146160e+00, 9.99950166e-01, 1.07116176e+00,
       1.07345534e+00, 1.04935645e+00, 1.01015089e+00, 1.03371751e+00,
       6.54338544e+04, 6.39443098e+04, 6.25215503e+04, 5.72229826e+04,
       5.45746221e+04, 5.33084779e+04, 1.51993804e+04, 1.80845825e+04,
       1.44245395e+04, 1.38536166e+04, 1.33906631e+04, 1.61330268e+04])}))],
         verbose=False)

In [44]:
pipe_op = BADSPipeLine("./0405_Piepeline.pkl")
pipe_op.save(pipe)
pipe_op.load()
pipe_op.transform(test)

train = pipe_op.transform(train)
valid = pipe_op.transform(valid)
test = pipe_op.transform(test)

Transform MissingData Method : mean
Transform MissingData Method : most_frequent
Transform NumericData
Transform MissingData Method : mean
Transform MissingData Method : most_frequent
Transform NumericData
Transform MissingData Method : mean
Transform MissingData Method : most_frequent
Transform NumericData
Transform MissingData Method : mean
Transform MissingData Method : most_frequent
Transform NumericData


In [45]:
pipe[-1].trans_info

{'fill_value': array([ 1.67484323e+05,  3.54752228e+01, -1.70900886e-02, -1.25958973e-01,
        -1.65824567e-01, -2.16004663e-01, -2.64603095e-01, -2.86467414e-01,
         5.10673010e+04,  4.95652237e+04,  4.70681142e+04,  4.32569520e+04,
         4.04164970e+04,  3.88574464e+04,  5.69172658e+03,  5.81409563e+03,
         5.13448505e+03,  4.83411333e+03,  4.76632123e+03,  5.24146976e+03]),
 'norm_mean': array([ 1.67484323e+05,  3.54752228e+01, -1.70900886e-02, -1.25958973e-01,
        -1.65824567e-01, -2.16004663e-01, -2.64603095e-01, -2.86467414e-01,
         5.10673010e+04,  4.95652237e+04,  4.70681142e+04,  4.32569520e+04,
         4.04164970e+04,  3.88574464e+04,  5.69172658e+03,  5.81409563e+03,
         5.13448505e+03,  4.83411333e+03,  4.76632123e+03,  5.24146976e+03]),
 'norm_std': array([1.29747662e+05, 8.24146160e+00, 9.99950166e-01, 1.07116176e+00,
        1.07345534e+00, 1.04935645e+00, 1.01015089e+00, 1.03371751e+00,
        6.54338544e+04, 6.39443098e+04, 6.25215503e+0

In [46]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
category_info = {}

class CategroyLabelEncoding(object) :
    def __init__(self , fac_var , in_var) :
        self.fac_var = fac_var
        self.in_var = in_var
        self.category_info = {}
    
    def fit(self, data) :
        for feat in self.fac_var:
            findIDX = [idx for idx, i in enumerate(self.in_var) if i == feat][0]
            lbe = LabelEncoder()
            lbe.fit(data[feat].values)
            diz_map_train = dict(zip(lbe.classes_, lbe.transform(lbe.classes_) + 1))
            self.category_info[feat] = diz_map_train
        return "label encoding"
    
    def transform(self,data) :
        for feat in self.fac_var:
            findIDX = [idx for idx, i in enumerate(self.in_var) if i == feat][0]
            current_key = list(self.category_info[feat].keys())
            remain_set = set(data[feat]).difference(current_key)
            if remain_set == set() :
                pass
            else :
                for i in remain_set :  ## if not in train impose 0
                    self.category_info[feat].update({i : 0})
#         data[feat] = [diz_map_train[i] for i in data[feat].values]
        print(self.category_info)
        data2 = data.replace(self.category_info)
        return data2

    
testclass = CategroyLabelEncoding(fac_var , in_var)
testclass.fit(train)
train = testclass.transform(train)

{'SEX': {'1.0': 1, '2.0': 2, 'nan': 3}, 'EDUCATION': {'0.0': 1, '1.0': 2, '2.0': 3, '3.0': 4, '4.0': 5, '5.0': 6, '6.0': 7, 'nan': 8}, 'MARRIAGE': {'0.0': 1, '1.0': 2, '2.0': 3, '3.0': 4, 'nan': 5}}


In [47]:
valid = testclass.transform(valid)
test = testclass.transform(test)

{'SEX': {'1.0': 1, '2.0': 2, 'nan': 3}, 'EDUCATION': {'0.0': 1, '1.0': 2, '2.0': 3, '3.0': 4, '4.0': 5, '5.0': 6, '6.0': 7, 'nan': 8}, 'MARRIAGE': {'0.0': 1, '1.0': 2, '2.0': 3, '3.0': 4, 'nan': 5}}
{'SEX': {'1.0': 1, '2.0': 2, 'nan': 3}, 'EDUCATION': {'0.0': 1, '1.0': 2, '2.0': 3, '3.0': 4, '4.0': 5, '5.0': 6, '6.0': 7, 'nan': 8}, 'MARRIAGE': {'0.0': 1, '1.0': 2, '2.0': 3, '3.0': 4, 'nan': 5}}


In [48]:
cat_info = testclass.category_info

In [49]:
cat_info

{'SEX': {'1.0': 1, '2.0': 2, 'nan': 3},
 'EDUCATION': {'0.0': 1,
  '1.0': 2,
  '2.0': 3,
  '3.0': 4,
  '4.0': 5,
  '5.0': 6,
  '6.0': 7,
  'nan': 8},
 'MARRIAGE': {'0.0': 1, '1.0': 2, '2.0': 3, '3.0': 4, 'nan': 5}}

In [50]:
Train_X_np = train[in_var].values
Train_y = train[target_var[0]]
Test_X_np = test[in_var].values
Test_y = test[target_var[0]]

In [51]:
train[in_var].isna().sum()

LIMIT_BAL    0
SEX          0
EDUCATION    0
MARRIAGE     0
AGE          0
PAY_0        0
PAY_2        0
PAY_3        0
PAY_4        0
PAY_5        0
PAY_6        0
BILL_AMT1    0
BILL_AMT2    0
BILL_AMT3    0
BILL_AMT4    0
BILL_AMT5    0
BILL_AMT6    0
PAY_AMT1     0
PAY_AMT2     0
PAY_AMT3     0
PAY_AMT4     0
PAY_AMT5     0
PAY_AMT6     0
dtype: int64

In [52]:
for idx , name in enumerate(in_var) :
    if name in list(cat_info.keys()) :
        cat_info[idx] = cat_info.pop(name)
cat_info

{1: {'1.0': 1, '2.0': 2, 'nan': 3},
 2: {'0.0': 1,
  '1.0': 2,
  '2.0': 3,
  '3.0': 4,
  '4.0': 5,
  '5.0': 6,
  '6.0': 7,
  'nan': 8},
 3: {'0.0': 1, '1.0': 2, '2.0': 3, '3.0': 4, 'nan': 5}}

In [53]:
import tensorflow as tf
import numpy as np
import pandas as pd , sys
import matplotlib.pyplot as plt
import scikitplot as skplt
from IPython.display import clear_output

In [54]:
row , dim = Train_X_np.shape
print(row,dim)
target_n = 2 
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape = [ None , dim])
y = tf.placeholder(tf.float32, shape = [ None , 1])
DropoutRate = tf.placeholder(tf.float32, name ="dropoutRate")
training = tf.placeholder_with_default(False, shape=[], name="training")
global_step = tf.get_variable('global_step', [], initializer=tf.constant_initializer(0), trainable=False)
batch_prob  = tf.placeholder(tf.bool)
batch_size = tf.placeholder(tf.int64)

30000 23


In [55]:
# i = 1 
# split = tf.slice(X, [0, i], [-1, 1])
# split =tf.cast(split, dtype=tf.int32)
# split = tf.reshape(split,(-1,))
# size = max(list(cat_info[i].values())) + 1
# emb = 4
# tf.keras.layers.Embedding(size,emb)(split)

In [56]:
cat_info

{1: {'1.0': 1, '2.0': 2, 'nan': 3},
 2: {'0.0': 1,
  '1.0': 2,
  '2.0': 3,
  '3.0': 4,
  '4.0': 5,
  '5.0': 6,
  '6.0': 7,
  'nan': 8},
 3: {'0.0': 1, '1.0': 2, '2.0': 3, '3.0': 4, 'nan': 5}}

In [57]:
total = np.arange(len(in_var))
fac_idx = [1,2,3]
split = list(set(np.array(fac_idx)) | set(np.array(fac_idx ) + 1))
lists = np.split(total , split)
lists

[array([0]),
 array([1]),
 array([2]),
 array([3]),
 array([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
        21, 22])]

In [58]:
index_store = []
for idx , i in enumerate(lists) :
    if len(i) == 1 :
        index_store.append([i[0],1])
    else :
        index_store.append([i[0],len(i) ] ) 
index_store

[[0, 1], [1, 1], [2, 1], [3, 1], [4, 19]]

In [59]:
def RepresentationLayer(X, emb_dim,index_store,category_info, type) :
    inputs = []
    embs = {}
    for index in index_store :
        split = tf.slice(X, [0, index[0]], [-1, index[1]])
        if index[0] in list(category_info.keys()) :
            split =tf.cast(split, dtype=tf.int32)
            split = tf.reshape(split,(-1,))
            size = max(list(category_info[index[0]].values())) + 1
            if type == "embedding" :    
                Cat = tf.keras.layers.Embedding(size,emb_dim)(split)
                embs[index[0]] = Cat
            elif type == "onehot" :
                Cat = tf.one_hot(split ,depth=size)
            else :
                raise Exception(f"No Valid Type : {type}, Please Change the type to onehot or embedding")
            inputs.append(Cat)
        else :
            inputs.append(split)
#     group_key = list(category_info.keys())
#     group_vec = []
#     for g_k in group_key :
#         new_group_key = group_key[:]
#         new_group_key.remove(g_k)
#         dots = [tf.keras.layers.Dot(axes=1)([embs[k], embs[g_k]]) for k in new_group_key]
#         dot_product = tf.keras.layers.Average()(dots)
#         group_vec.append(dot_product)
#     group_matrix = tf.concat(group_vec, axis=1, name='groupvec_')
# #     group_matrix  = tf.nn.dropout(group_matrix, 0.5)
#     group_matrix = tf.layers.batch_normalization(group_matrix,
#                                                  center=True, scale=True, training=True)
#     inputs.append(group_matrix )
    concatenated_layer = tf.concat(inputs, axis=1, name='concatenate')
    return concatenated_layer
def Layer(X , reuse = False) :
    with tf.variable_scope("Final",reuse=reuse):
        W_1 = tf.get_variable("w_1",shape=[X.get_shape()[1],70])
        B_1 = tf.get_variable("b_1",shape=[70])
        W_2 = tf.get_variable("w_2",shape=[70,40])
        B_2 = tf.get_variable("b_2",shape=[40])
        W_3 = tf.get_variable("w_3",shape=[40,15])
        B_3 = tf.get_variable("b_3",shape=[15])
        finalW = tf.get_variable("final_w",shape=[15,1])
        finalB = tf.get_variable("final_b",shape=[1])
        layer1 = tf.matmul(X, W_1) + B_1
        layer1 = tf.layers.batch_normalization(layer1, 
                                               center=True, scale=True, training=batch_prob)
        layer1 = tf.nn.selu(layer1)
        layer1 = tf.nn.dropout(layer1, keep_prob=DropoutRate)
        layer2 = tf.matmul(layer1, W_2) + B_2
        layer2 = tf.layers.batch_normalization(layer2, 
                                               center=True, scale=True, training=batch_prob)
        layer2 = tf.nn.selu(layer2)
        layer2 = tf.nn.dropout(layer2, keep_prob=DropoutRate)
        layer3 = tf.matmul(layer2, W_3) + B_3
        layer3 = tf.layers.batch_normalization(layer3, 
                                               center=True, scale=True, training=batch_prob)
        layer3 = tf.nn.selu(layer3)
        layer3 = tf.nn.dropout(layer3, keep_prob=DropoutRate)
        logit = tf.matmul(layer3, finalW) + finalB
    return logit

In [60]:
data_tuple = (X,y)
class_dataset = tf.data.Dataset.from_tensor_slices(data_tuple)
class_dataset = class_dataset.shuffle(buffer_size= 30000,)
class_dataset = class_dataset.batch(batch_size=batch_size, drop_remainder=True)
iter = class_dataset.make_initializable_iterator()
feature_x , label_y = iter.get_next()

In [61]:
FeatureX = RepresentationLayer(feature_x , 4 , index_store , cat_info, type="onehot")

In [62]:
logit = Layer(FeatureX)
prob= tf.nn.sigmoid(logit)
auc_value , update_auc = tf.metrics.auc(label_y , prob , curve="ROC")

In [63]:
class_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "Final") 
totalvars = tf.trainable_variables()

In [64]:
learning_rate = tf.train.cosine_decay_restarts(1e-4, global_step,
                                               first_decay_steps=100, t_mul=1.5,m_mul=0.9, alpha=0.0)
loss2 = tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(labels = label_y ,
                                                                logits=logit,pos_weight=1.5))
L2= []
for v in totalvars :
    L2.append(tf.nn.l2_loss(v))
L2Regularizer= tf.add_n(L2)  * 0.01
loss2 +=L2Regularizer
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    solver2 = tf.train.AdamOptimizer(learning_rate= learning_rate).minimize(loss2 ,var_list = totalvars)

In [65]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset
import matplotlib.patches as mpatches
from mpl_toolkits.axes_grid1.inset_locator import (inset_axes, TransformedBbox,
                                                   BboxPatch, BboxConnector)
def my_mark_inset(parent_axes, inset_axes, loc1a=1, loc1b=1, loc2a=2, loc2b=2, **kwargs):
    rect = TransformedBbox(inset_axes.viewLim, parent_axes.transData)
    pp = BboxPatch(rect, fill=False, **kwargs)
    parent_axes.add_patch(pp)
    p1 = BboxConnector(inset_axes.bbox, rect, loc1=loc1a, loc2=loc1b, **kwargs)
    inset_axes.add_patch(p1)
    p1.set_clip_on(False)
    p2 = BboxConnector(inset_axes.bbox, rect, loc1=loc2a, loc2=loc2b, **kwargs)
    inset_axes.add_patch(p2)
    p2.set_clip_on(False)
    return pp, p1, p2
def subplotting(ax , store , x , y ,cond={}, **kwargs) :
    ax.plot(store[x],store[y],**kwargs)
    if "ylabel" in cond :
        ax.set_ylabel(cond["ylabel"], fontsize= 10)
    if "xlabel" in cond :
        ax.set_xlabel(cond["xlabel"], fontsize= 10)
    if "xlim" in cond :
        ax.set_xlim(cond["xlim"])
    if "ylim" in cond :
        ax.set_ylim(cond["ylim"])
    if "title" in cond :
        ax.set_title(cond["title"], fontsize= 15)
    ax.legend()
    return ax
## version3
def vis_onlysl(store:dict, path:str,title:str) :
    clear_output()
    fig = plt.figure(figsize=(10, 10))
    gs = GridSpec(nrows=2, ncols=2)
    ax1 = fig.add_subplot(gs[0:1, 0])
    ax2 = fig.add_subplot(gs[0:1, 1])
    ax3 = fig.add_subplot(gs[1, 0])
    ax4 = fig.add_subplot(gs[1, 1])
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.99, 
            top=0.9, wspace=0.3, hspace=0.2)
    ax1.plot(store["epoch"],store["slloss"],label = "SLloss", color='c')
    ax1.set_xlabel('Epoch')
    select_sl= np.argmin(store["slloss"])
    ax1.vlines(store["epoch"][select_sl],
               np.min(store["slloss"]),np.max(store["slloss"]),
               label='Best', color='c')
    a =store["slloss"][-1]
    ax1.set_title(f'Loss : {a:.4f}')
    ax1.set_ylabel('Supervised', fontsize= 15)
    ax1.legend()
    ax2.plot(store["epoch"],store["auc"],label = "Train auc")
    ax2.plot(store["epoch"],store["teauc"],label = "Test auc")
    ax2.set_ylabel('AUC', fontsize= 15)
    ax2.set_xlabel('Epoch')
    select= np.argmax(store["teauc"])
    msg = f"test idx : {store['epoch'][select]}, maximum : {store['teauc'][select]*100:.2f}"
    ax2.set_title(msg)             
#     ax2.set_ylim(0.7,store['auc'][select]+0.1)
    ax2.legend()
    check_n = 4
    if len(store['epoch']) > check_n :
        axins = inset_axes(ax2, "100%", "100%", 
                           bbox_to_anchor=[0.36, .3, .5, .4],
                       bbox_transform=ax2.transAxes, borderpad=0)
        axins.plot(store['epoch'], store['auc'])
        axins.plot(store['epoch'], store['teauc'])
        maximum = np.max(store['auc'][-check_n:] + store['teauc'][-check_n:])
        minumum = np.min(store['auc'][-check_n:] + store['teauc'][-check_n:])
        xlims = (store['epoch'][-check_n],store['epoch'][-1])
        ylims = (minumum, maximum)
        axins.set(xlim=xlims, ylim=ylims)
        my_mark_inset(ax2, axins, loc1a=2, loc1b=3, loc2a=4, loc2b=4, fc="none", ec="0.5") # 
#     msg = f"Epoch : {epoch[-1]}, Loss : {loss[-1]:.3f}, Auc : {aucs[-1]:.3f}"
#     skplt.metrics.plot_ks_statistic(store["train_y"], store["train_prob"], 
#                                 ax = ax3 ,
#                                 title = "[Train] KS Static PLOT")
#     skplt.metrics.plot_ks_statistic(store["test_y"], store["test_prob"], 
#                                 ax = ax4 ,
#                                 title = "[Test] KS Static PLOT")
    sns.boxplot(x="t", y="prob", data=store["train_pd"] , ax = ax3)
    ax3.set_title("train" , fontsize= 15)
    sns.boxplot(x="t", y="prob", data=store["test_pd"] , ax = ax4)
    ax4.set_title("test" , fontsize= 15)
    plt.suptitle(title)
    plt.savefig(path)
    plt.show()

In [ ]:
config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True
Epoch = 100000
oversample = list(np.arange(len(Train_y))) + 2 * list(np.where(Train_y.values == 1)[0])
from matplotlib.gridspec import GridSpec
from sklearn.metrics import auc , roc_auc_score
with tf.Session(config=config) as sess:
    init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    sess.run(init)
    store = {"epoch" : [] ,  "slloss" : [],
             "auc" : [], "teauc" : []}
    store["train_pd"] = []
    store["test_pd"] = []
    for i in range(Epoch):    
        batchSlLoss = []
        permutation_idx = np.random.permutation(oversample)
        sess.run(iter.initializer , 
                 feed_dict= {X : Train_X_np[permutation_idx],
                             y : Train_y.values.reshape(-1,1)[[permutation_idx]],
                             batch_size : 300,
                            }) # switch to train dataset
        while True :
            try :
                ### Supervised Learning
                result  = sess.run([solver2 , loss2],
                                   feed_dict={global_step:i, 
                                              batch_prob :True,
                                              DropoutRate : 0.8
                                             })
                batchSlLoss.append(result[1])
            except tf.errors.OutOfRangeError :
                break
        mslloss = np.mean(batchSlLoss)
        print(f"Epoch : {i}, SLLoss : {mslloss:.3f}" , end = "\r")
        if (i % 100 == 0) :
            sess.run(iter.initializer , 
                     feed_dict= {X : Train_X_np,
                     y : Train_y.values.reshape(-1,1),
                     batch_size : len(Train_X_np),}) 
            tr_result  = sess.run([prob,label_y],
                                  feed_dict={batch_prob :False,
                                             DropoutRate : 1.0
                                            })
            sess.run(iter.initializer , 
                     feed_dict= {X : Test_X_np,
                     y : Test_y.values.reshape(-1,1),
                     batch_size : len(Test_X_np),}) 
            result  = sess.run([prob,label_y],
                               feed_dict={batch_prob :False,
                                          DropoutRate : 1.0
                                         })
            store["epoch"].append(i)
            store["slloss"].append(mslloss)
            store["train_pd"] = pd.DataFrame(np.concatenate(tr_result,axis=1),
                                             columns=["prob","t"])
            store["test_pd"] = pd.DataFrame(np.concatenate(result,axis=1),
                                             columns=["prob","t"])
            store["auc"].append(roc_auc_score(store["train_pd"]["t"].values,store["train_pd"]["prob"].values))
            store["teauc"].append(roc_auc_score(store["test_pd"]["t"].values,store["test_pd"]["prob"].values))
#             store["train_prob"] =  np.concatenate((1-tr_result[2],tr_result[2]),axis=1)
#             store["test_prob"] =  np.concatenate((1-result[2],result[2]),axis=1)
            vis_onlysl(store, "./0405_SL_FeatureX.png", title="Train SL")
            clear_output()

/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


In [ ]:
for key, value in  category_info.items() :
    a[key] = pd.Categorical(a[key],list(value.values()))
pd.get_dummies(a,columns=fac_var).columnsumns.tolist()

In [86]:
pd.get_dummies(a)

,SEX_1,SEX_2,SEX_3,EDUCATION_1,EDUCATION_2,EDUCATION_3,EDUCATION_4,EDUCATION_5,EDUCATION_6,EDUCATION_7,EDUCATION_8,MARRIAGE_1,MARRIAGE_2,MARRIAGE_3,MARRIAGE_4,MARRIAGE_5


In [70]:
train.replace(category_info)

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month,sep_idx
0,20000.0,2.0,2.0,1.0,24.0,2.0,2.0,-1.0,-1.0,-2.0,...,0.0,NaN,0.0,689.0,0.0,0.0,NaN,0.0,1.0,1
1,120000.0,2.0,nan,2.0,26.0,-1.0,2.0,0.0,NaN,0.0,...,3455.0,3261.0,0.0,1000.0,1000.0,NaN,0.0,2000.0,1.0,1
2,90000.0,2.0,2.0,2.0,NaN,0.0,0.0,0.0,0.0,0.0,...,14948.0,15549.0,1518.0,1500.0,NaN,1000.0,1000.0,NaN,0.0,1
3,50000.0,2.0,2.0,1.0,37.0,0.0,0.0,0.0,0.0,0.0,...,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,NaN,1000.0,0.0,1
4,50000.0,1.0,nan,nan,37.0,0.0,0.0,0.0,0.0,0.0,...,NaN,20024.0,2500.0,1815.0,657.0,1000.0,NaN,800.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,50000.0,1.0,2.0,1.0,44.0,1.0,2.0,2.0,2.0,0.0,...,22676.0,14647.0,2300.0,NaN,0.0,517.0,503.0,585.0,0.0,0
29996,240000.0,1.0,1.0,2.0,30.0,-2.0,-2.0,-2.0,-2.0,-2.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0
29997,80000.0,1.0,nan,nan,34.0,2.0,2.0,NaN,2.0,2.0,...,82607.0,NaN,NaN,3500.0,0.0,7000.0,0.0,4000.0,1.0,0
29998,150000.0,nan,3.0,2.0,43.0,NaN,-1.0,NaN,-1.0,NaN,...,5190.0,0.0,1837.0,3526.0,8998.0,129.0,0.0,0.0,0.0,0


In [71]:
#     for i in set(valid[feat]).difference(train[feat]):  ## if not in train impose 0
#         diz_map_train[i] = 0

#     for i in set(test[feat]).difference(train[feat]):  ## if not in train impose 0
#         diz_map_train[i] = 0
    
#     train[feat] = [diz_map_train[i] for i in train[feat].values]
#     valid[feat] = [diz_map_train[i] for i in valid[feat].values]
#     test[feat] = [diz_map_train[i] for i in test[feat].values]

In [72]:
data = train
category_info  = {}
for feat in fac_var:
    findIDX = [idx for idx, i in enumerate(in_var) if i == feat][0]
    lbe = LabelEncoder()
    lbe.fit(data[feat].values)
    diz_map_train = dict(zip(lbe.classes_, lbe.transform(lbe.classes_) + 1))
    category_info[feat] = diz_map_train
    print(category_info)
train.replace(category_info)

{'SEX': {'1.0': 1, '2.0': 2, 'nan': 3}}
{'SEX': {'1.0': 1, '2.0': 2, 'nan': 3}, 'EDUCATION': {'0.0': 1, '1.0': 2, '2.0': 3, '3.0': 4, '4.0': 5, '5.0': 6, '6.0': 7, 'nan': 8}}
{'SEX': {'1.0': 1, '2.0': 2, 'nan': 3}, 'EDUCATION': {'0.0': 1, '1.0': 2, '2.0': 3, '3.0': 4, '4.0': 5, '5.0': 6, '6.0': 7, 'nan': 8}, 'MARRIAGE': {'0.0': 1, '1.0': 2, '2.0': 3, '3.0': 4, 'nan': 5}}


,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month,sep_idx
0,20000.0,2,3,2,24.0,2.0,2.0,-1.0,-1.0,-2.0,...,0.0,NaN,0.0,689.0,0.0,0.0,NaN,0.0,1.0,1
1,120000.0,2,8,3,26.0,-1.0,2.0,0.0,NaN,0.0,...,3455.0,3261.0,0.0,1000.0,1000.0,NaN,0.0,2000.0,1.0,1
2,90000.0,2,3,3,NaN,0.0,0.0,0.0,0.0,0.0,...,14948.0,15549.0,1518.0,1500.0,NaN,1000.0,1000.0,NaN,0.0,1
3,50000.0,2,3,2,37.0,0.0,0.0,0.0,0.0,0.0,...,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,NaN,1000.0,0.0,1
4,50000.0,1,8,5,37.0,0.0,0.0,0.0,0.0,0.0,...,NaN,20024.0,2500.0,1815.0,657.0,1000.0,NaN,800.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,50000.0,1,3,2,44.0,1.0,2.0,2.0,2.0,0.0,...,22676.0,14647.0,2300.0,NaN,0.0,517.0,503.0,585.0,0.0,0
29996,240000.0,1,2,3,30.0,-2.0,-2.0,-2.0,-2.0,-2.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0
29997,80000.0,1,8,5,34.0,2.0,2.0,NaN,2.0,2.0,...,82607.0,NaN,NaN,3500.0,0.0,7000.0,0.0,4000.0,1.0,0
29998,150000.0,3,4,3,43.0,NaN,-1.0,NaN,-1.0,NaN,...,5190.0,0.0,1837.0,3526.0,8998.0,129.0,0.0,0.0,0.0,0


In [43]:
category_info

{'SEX': {'1.0': 1, '2.0': 2, 'nan': 3},
 'EDUCATION': {'0.0': 1,
  '1.0': 2,
  '2.0': 3,
  '3.0': 4,
  '4.0': 5,
  '5.0': 6,
  '6.0': 7,
  'nan': 8},
 'MARRIAGE': {'0.0': 1, '1.0': 2, '2.0': 3, '3.0': 4, 'nan': 5}}

In [47]:
set(list(category_info["SEX"].keys())).difference(list(category_info["SEX"].keys())) == set()

True

set()

In [33]:
for i in set(np.array(["4.0","2.0"])).difference((list(category_info[1].keys()))) :
    category_info[1].update({i : 0})

In [34]:
category_info

{1: {'1.0': 1, '2.0': 2, 'nan': 3, '4.0': 0},
 2: {'0.0': 1,
  '1.0': 2,
  '2.0': 3,
  '3.0': 4,
  '4.0': 5,
  '5.0': 6,
  '6.0': 7,
  'nan': 8},
 3: {'0.0': 1, '1.0': 2, '2.0': 3, '3.0': 4, 'nan': 5}}

'Save Pipeline'

'Load Pipeline'

Transform MissingData Method : mean
Transform MissingData Method : most_frequent
Transform NumericData


,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month,sep_idx
0,14945.0,0.096462,1.0,3.0,1.0,0.913039,-9.829589e-01,2.591166e-17,-7.770938e-01,-0.747120,...,-7.365786e-01,-0.603421,6.572816e-01,-0.061660,0.000000,-0.333206,0.145152,0.415764,0.0,0
1,14947.0,-0.751338,2.0,2.0,1.0,-0.543013,-1.040886e-17,1.175910e-01,1.544774e-01,0.205845,...,5.243005e-01,0.523998,5.983762e-17,-0.168049,-0.190889,-0.132392,0.000000,-0.163235,0.0,0
2,14948.0,-0.905483,2.0,2.0,2.0,-1.635053,1.709094e-02,2.591166e-17,2.017620e+00,0.000000,...,-2.426677e-01,-0.233236,-5.807648e-02,-0.321495,-0.272763,-0.240667,-0.285521,-0.254104,0.0,0
3,14949.0,2.562787,1.0,nan,2.0,-0.664351,1.017141e+00,1.175910e-01,1.544774e-01,0.205845,...,1.032990e-01,-0.085680,-4.307587e-02,2.289625,0.000000,0.806352,-0.206586,-0.262906,0.0,0
4,14950.0,0.250607,2.0,1.0,2.0,0.000000,-9.829589e-01,-8.159748e-01,-7.770938e-01,-0.747120,...,-7.405731e-01,-0.728917,5.983762e-17,0.000000,-0.355955,-0.348942,0.000000,-0.324891,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5999,29989.0,0.635970,1.0,1.0,nan,-0.179000,-1.040886e-17,1.175910e-01,1.544774e-01,0.205845,...,2.466137e+00,2.641785,3.902019e+00,0.000000,0.268744,0.084158,0.166809,0.047575,0.0,0
6000,29990.0,-0.134756,1.0,nan,2.0,-0.057662,-9.829589e-01,-8.159748e-01,-7.770938e-01,-0.747120,...,-1.333213e-16,-0.728917,2.212112e-01,-0.321495,-0.301672,-0.348942,-0.355944,-0.324891,0.0,0
6001,29995.0,-0.674265,1.0,2.0,nan,-0.179000,2.017191e+00,1.984723e+00,2.585629e-17,0.000000,...,-1.333213e-16,0.793505,8.607413e-02,0.000000,0.000000,0.156341,-0.355944,-0.076952,1.0,0
6002,29997.0,-0.134756,1.0,3.0,nan,0.913039,-1.040886e-17,2.591166e-17,-7.770938e-01,-0.747120,...,-6.454740e-01,0.000000,5.983762e-17,-0.126522,0.267843,-0.339631,-0.355944,-0.324891,0.0,0


In [13]:
from sklearn.externals import joblib
joblib.dump(pipe , "model.joblib")

PicklingError: Can't pickle <class 'PreProcessingBASD_v2.MissingHandling'>: it's not the same object as PreProcessingBASD_v2.MissingHandling